# Technical Factor - Stochastic Oscillator (KDJ)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        (close - m_min(low, 9)) / (m_max(high, 9) - m_min(low, 9)) * 100 AS RSV,
        m_avg(RSV, 9) AS K,
        m_avg(K, 9)   AS D,
        3 * K - 2 * D AS J,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(K > D AND m_lag(K, 1) <= m_lag(D, 1), 1, 0) AS TRBY1,
        IF(K < D AND m_lag(K, 1) >= m_lag(D, 1), 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(J > m_lag(J, 1) AND m_lag(J, 1) >= m_lag(J, 2), 1, 0) AS MTBY1,
        IF(J < m_lag(J, 1) AND m_lag(J, 1) <= m_lag(J, 2), 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(J < 0 AND (J - m_lag(J, 1)) > 0, 1, 0) AS RVBY1,
        IF(J > 100 AND (J - m_lag(J, 1)) < 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(J, 1) <= 80 AND J > 80, 1, 0) AS BKBY1,
        IF(m_lag(J, 1) >= 20 AND J < 20, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        RSV,
        K,
        D,
        J,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,RSV,K,D,J,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-02-13,600519.SH,86.456672,32.557988,33.271493,31.130978,0,0,0,0,0,0,0,0
1,2025-02-14,600519.SH,97.402260,42.317546,32.721121,61.510395,1,0,0,0,0,0,0,0
2,2025-02-17,600519.SH,75.402759,49.284836,33.396386,81.061734,0,0,1,0,0,0,1,0
3,2025-02-18,600519.SH,78.961777,57.856646,35.834970,101.899999,0,0,1,0,0,0,0,0
4,2025-02-19,600519.SH,95.653598,66.896992,39.597676,121.495622,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,2025-12-25,600519.SH,81.151567,77.095454,55.465109,120.356144,0,0,0,1,0,1,0,0
215,2025-12-26,600519.SH,81.009583,77.721624,61.231543,110.701786,0,0,0,1,0,1,0,0
216,2025-12-29,600519.SH,31.550236,73.634646,65.726366,89.451205,0,0,0,1,0,0,0,0
217,2025-12-30,600519.SH,11.111111,65.105674,68.320075,58.676873,0,1,0,1,0,0,0,0
